In [2]:
# Convolutional Neural Network
import torch
import torchvision
import torchvision.transforms as transforms

In [ ]:

trainset = torchvision.datasets.SVHN(root='./datasets', 
                                     split='train', 
                                     download=True, 
                                     transform=transforms.ToTensor())